<a href="https://colab.research.google.com/github/Alignment-Lab-AI/AutoMeticAssistant/blob/main/FastLanguageModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip

--2023-12-26 23:16:44--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.226.34.83, 13.226.34.53, 13.226.34.7, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.226.34.83|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1523785255 (1.4G) [application/zip]
Saving to: ‘crawl-300d-2M.vec.zip’

crawl-300d-2M.vec.z 100%[===================>]   1.42G  78.4MB/s    in 15s     

2023-12-26 23:16:59 (99.9 MB/s) - ‘crawl-300d-2M.vec.zip’ saved [1523785255/1523785255]



In [ ]:
!unzip /content/crawl-300d-2M.vec.zip

Archive:  /content/crawl-300d-2M.vec.zip
  inflating: crawl-300d-2M.vec       

In [ ]:
!pip install transformers  fasttext datasets

In [ ]:
!git clone https://github.com/facebookresearch/fastText.git

In [ ]:
cd fastText

In [ ]:
!sudo python setup.py install

In [ ]:
!make

In [ ]:
cd ./../

In [ ]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("Open-Orca/OpenOrca")['all']
dataset = dataset.select(range(5000))

with open('./train.txt', 'w', encoding='utf-8') as file:
    for conversation in dataset:
        # Initialize a temporary string to hold the conversation text
        conversation_text = ''
        for turn in conversation['conversation']:
            # Check the 'from' field to determine if it's a prompt or a response
            if turn['from'] in ['system', 'human']:
                # Add the prompt to the conversation text
                conversation_text += turn['value'] + '\n'
            elif turn['from'] == 'gpt':
                # Add the response to the conversation text
                conversation_text += turn['value'] + '\n\n'
        # Write the processed conversation to the file
        file.write(conversation_text)


In [ ]:

def load_embeddings(file_path):
    embeddings = {}
    with open(file_path, 'r', encoding='utf-8', newline='\n', errors='ignore') as f:
        n=0
        for line in f:
            n=n+1
            if n<50000:
             values = line.strip().split(' ')
             word = values[0]
             vector = [float(val) for val in values[1:]]
             embeddings[word] = vector
    return embeddings
def reduce_dimensionality(embeddings, n_components=32):
    reduced_embeddings = {}
    for i, (word, vector) in enumerate(embeddings.items()):
        if i >= 50000:
            break
        reduced_vector = vector[:n_components]
        reduced_embeddings[word] = reduced_vector
    return reduced_embeddings

def save_embeddings(embeddings, output_file):
    with open(output_file, 'w', encoding='utf-8') as f:
        for word, vector in embeddings.items():
            vector_str = ' '.join(str(val) for val in vector)
            f.write(f'{word} {vector_str}\n')

input_file = '/content/crawl-300d-2M.vec.zip'
output_file = 'reduced_embeddings_32d_50k.vec'
embeddings = load_embeddings(input_file)
reduced_embeddings = reduce_dimensionality(embeddings, n_components=32)
save_embeddings(reduced_embeddings, output_file)


In [ ]:
# Define a function to load the first 50,000 words
def load_top_words(filename, limit=50000):
    vocab = {}
    with open(filename, 'r', encoding='utf-8', newline='\n', errors='ignore') as f:
        for line in f:
            tokens = line.rstrip().split(' ')
            word = tokens[0]
            if word not in vocab:
                vocab[word] = len(vocab) + 1  # assign an index to each word
            if len(vocab) >= limit:
                break
    return vocab

# Load the top 50,000 words
vocab = load_top_words('/content/reduced_embeddings_32d_50k.vec')


In [ ]:

# Process your text file
data = []
with open('./train.txt', 'r') as file:
    for line in file:
        words = line.strip().split()
        for i, word in enumerate(words):
            # Check if the word is in the vocabulary
            if word not in vocab:
                word = '<unk>'
            if i > 0:
                # Use double quotes for the f-string and single quotes inside it
                label = f"__label__{words[i-1]}"
                # Use  words or '<unk>' if they are not in vocab
                context = ' '.join(words)
                labelled_example = f"{label} {context}"
                data.append(labelled_example)

# Write the data to the output file
with open('./ft.txt', 'w') as file:
    file.write('\n'.join(data))


In [ ]:
def edit_first_line(file_path, new_first_line):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # Substitua a primeira linha
    lines[0] = new_first_line + '\n'

    with open(file_path, 'w', encoding='utf-8') as file:
        file.writelines(lines)

# Caminho para o arquivo "crawl-300d-2M.vec"
file_path = '/content/reduced_embeddings_32d_50k.vec'

# Nova primeira linha
new_first_line = '50000 32'

# Edite a primeira linha
edit_first_line(file_path, new_first_line)


GOOD QUALITY = 500 epochs
NORMAL QUALITY  = 5 epochs

In [ ]:
!./fastText/fasttext  supervised -input ft.txt -output model -loss ns -dim 32 -lr .5 -epoch 5   -pretrainedVectors /content/reduced_embeddings_32d_50k.vec

In [ ]:
import fasttext
import numpy as np

# Assuming you have a trained FastText model
model = fasttext.load_model('model.bin')

def apply_repetition_penalty(labels, probabilities, used_labels, penalty_scale=1.9):
    """
    Applies a repetition penalty to reduce the probability of already used labels.

    :param labels: List of possible labels.
    :param probabilities: Corresponding list of probabilities.
    :param used_labels: Set of labels that have already been used.
    :param penalty_scale: Scale of the penalty to be applied.
    :return: Adjusted probabilities.
    """
    adjusted_probabilities = probabilities.copy()
    for i, label in enumerate(labels):
        if label in used_labels:
            adjusted_probabilities[i] /= penalty_scale
    # Normalize the probabilities to sum to 1 again
    adjusted_probabilities /= adjusted_probabilities.sum()
    return adjusted_probabilities

def predict_sequence(model, text, sequence_length=20, temperature=.9, penalty_scale=1.5):
    """
    Generates a sequence of labels using the FastText model with repetition penalty.

    :param model: Loaded FastText model.
    :param text: Initial text to start the prediction from.
    :param sequence_length: Desired length of the sequence.
    :param temperature: Temperature for sampling.
    :param penalty_scale: Scale of repetition penalty.
    :return: List of predicted labels.
    """
    used_labels = set()
    sequence = []

    for _ in range(sequence_length):
        # Predict the top k most probable labels
        labels, probabilities = model.predict(text, k=40)
        labels = [label.replace('__label__', '') for label in labels]
        probabilities = np.array(probabilities)

        # Adjust the probabilities with repetition penalty
        probabilities = apply_repetition_penalty(labels, probabilities, used_labels, penalty_scale)

        # Sampling according to the adjusted probabilities
        label_index = np.random.choice(range(len(labels)), p=probabilities)
        chosen_label = labels[label_index]

        # Add the chosen label to the sequence and to the set of used labels
        sequence.append(chosen_label)
        used_labels.add(chosen_label)

        # Update the text with the chosen label for the next prediction
        text += ' ' + chosen_label

    return sequence

# Example usage
generated_sequence = predict_sequence(model, "How do you create an opinion poll?", sequence_length=50)

print("How do you create an opinion poll?")
print(' '.join(generated_sequence))

generated_sequence = predict_sequence(model, "What is a language model for?", sequence_length=50)

print("What is a language model for?")
print(' '.join(generated_sequence))


